In [3]:
import numpy as np
from skimage.io import imshow
import matplotlib.pyplot as plt

from skimage.transform import resize
from skimage.io import imread
import os

def color_map(N=224, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap


def color_map_viz():
    labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'void']
    nclasses = 21
    row_size = 50
    col_size = 500
    cmap = color_map()
    array = np.empty((row_size*(nclasses+1), col_size, cmap.shape[1]), dtype=cmap.dtype)
    for i in range(nclasses):
        array[i*row_size:i*row_size+row_size, :] = cmap[i]
    array[nclasses*row_size:nclasses*row_size+row_size, :] = cmap[-1]

    imshow(array)
    plt.yticks([row_size*i+row_size/2 for i in range(nclasses+1)], labels)
    plt.xticks([])
    plt.show()

In [6]:
import numpy as np
from skimage.transform import resize
from skimage.io import imread
import os

def preprocess_and_save(image_path, mask_path, cmap, output_size=(224, 224), output_dir='output'):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load and resize image
    image = imread(image_path)
    image = resize(image, output_size, anti_aliasing=True)
    image /= 255.0  # Normalize to [0, 1]
    image_flattened = image.flatten()

    # Load and resize mask
    mask_image = imread(mask_path)
    mask_image = resize(mask_image, output_size, anti_aliasing=False, preserve_range=True)
    
    # Initialize 22-channel binary mask
    channels = np.zeros((*output_size, 22), dtype=np.float32)

    # Map each pixel to the correct channel
    for i in range(output_size[0]):
        for j in range(output_size[1]):
            color = np.round(mask_image[i, j]).astype(int)  # Ensure colors are integers
            for idx, val in enumerate(cmap):
                if np.array_equal(color, val):  # Strict equality check
                    try:
                        channels[i, j, idx] = 1
                    except:
                        print('ignore')
                    break

    mask_flattened = channels.flatten()

    # Save flattened image and mask to text files
    image_filename = os.path.join(output_dir, os.path.basename(image_path) + '_image.txt')
    mask_filename = os.path.join(output_dir, os.path.basename(mask_path) + '_mask.txt')

    np.savetxt(image_filename, image_flattened, fmt='%.6f')
    np.savetxt(mask_filename, mask_flattened, fmt='%d')

def process_dataset(image_dir, mask_dir, cmap):
    file_names = os.listdir(image_dir)
    for file_name in file_names:
        if file_name.endswith('.jpg'):
            mask_name = file_name[:-4] + '.png'  # Change extension for mask
            mask_path = os.path.join(mask_dir, mask_name)
            image_path = os.path.join(image_dir, file_name)

            # Check if the mask file exists
            if os.path.exists(mask_path):
                preprocess_and_save(image_path, mask_path, cmap)
            else:
                print(f"Mask for {file_name} not found, skipping...")

# Example usage
cmap = color_map()  # Assuming this function returns the correct RGB values for each class
image_dir = 'data/VOC2007/JPEGImages'
mask_dir = 'data/VOC2007/SegmentationClass'
process_dataset(image_dir, mask_dir, cmap)

Mask for 009108.jpg not found, skipping...
Mask for 002153.jpg not found, skipping...
Mask for 008180.jpg not found, skipping...
Mask for 009279.jpg not found, skipping...
Mask for 003254.jpg not found, skipping...
Mask for 001727.jpg not found, skipping...
Mask for 001121.jpg not found, skipping...
Mask for 005396.jpg not found, skipping...
Mask for 009051.jpg not found, skipping...
Mask for 007543.jpg not found, skipping...
Mask for 008190.jpg not found, skipping...
Mask for 000684.jpg not found, skipping...
Mask for 000545.jpg not found, skipping...
Mask for 009351.jpg not found, skipping...
Mask for 008169.jpg not found, skipping...
Mask for 001079.jpg not found, skipping...
Mask for 004706.jpg not found, skipping...
Mask for 001203.jpg not found, skipping...
Mask for 003337.jpg not found, skipping...
Mask for 009896.jpg not found, skipping...
Mask for 004950.jpg not found, skipping...
Mask for 002636.jpg not found, skipping...
Mask for 004220.jpg not found, skipping...
Mask for 00